<div style="background-color:#000;"><img src="pqn.png"></img></div><div><a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://www.pyquantnews.com/getting-started-with-python-for-quant-finance/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.</div>

## Library installation

Install the Interactive Brokers Python API package so we can create contracts and orders and talk to TWS/IB Gateway from a notebook. Instructions are here: https://interactivebrokers.github.io/

This only installs the client library. You still need Interactive Brokers TWS or IB Gateway running locally, with API access enabled and the socket port matching what we connect to below.

## Imports and setup

We import ibapi (EClient, EWrapper, Contract, Order) to build the socket client and the contract/order messages, plus threading to keep the network loop running and time to pace polling and shutdown.

In [ ]:
from ibapi.client import EClient
from ibapi.contract import Contract
from ibapi.order import Order
from ibapi.wrapper import EWrapper

In [ ]:
import threading
import time

Keeping imports minimal is deliberate for a first integration. It reduces “mystery plumbing” so when something fails (port, clientId, permissions), we know exactly where to look.

## Implement a minimal IB wrapper

Define a small IB API app that captures the essential callbacks for safe execution: we need a valid order ID and basic observability around order lifecycle events.

In [ ]:
class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)

    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextorderId = orderId
        print("The next valid order id is: ", self.nextorderId)

    def orderStatus(
        self,
        orderId,
        status,
        filled,
        remaining,
        avgFullPrice,
        permId,
        parentId,
        lastFillPrice,
        clientId,
        whyHeld,
        mktCapPrice,
    ):
        print(
            "orderStatus - orderid:",
            orderId,
            "status:",
            status,
            "filled",
            filled,
            "remaining",
            remaining,
            "lastFillPrice",
            lastFillPrice,
        )

    def openOrder(self, orderId, contract, order, orderState):
        print(
            "openOrder id:",
            orderId,
            contract.symbol,
            contract.secType,
            "@",
            contract.exchange,
            ":",
            order.action,
            order.orderType,
            order.totalQuantity,
            orderState.status,
        )

    def execDetails(self, reqId, contract, execution):
        print(
            "Order Executed: ",
            reqId,
            contract.symbol,
            contract.secType,
            contract.currency,
            execution.execId,
            execution.orderId,
            execution.shares,
            execution.lastLiquidity,
        )

This is the “boring on purpose” part professionals keep small and testable. If we don’t wire up nextValidId and orderStatus early, we end up with orders we can’t track or, worse, duplicated orders after reconnects.

Create tiny helper functions for the long-running event loop and for building a simple stock Contract message.

In [ ]:
def run_loop():
    app.run()

In [ ]:
def stock_contract(symbol, secType="STK", exchange="SMART", currency="USD"):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = secType
    contract.exchange = exchange
    contract.currency = currency
    return contract

Helpers like stock_contract keep the “contract vs order” split clear, which matches how IB actually thinks about execution. That mental model matters later when you move beyond stocks into futures, options, or routed venues.

## Connect and confirm order ID

Connect to TWS/IB Gateway and start the IB network loop on a background thread so our script can keep running while callbacks arrive asynchronously.

In [ ]:
app = IBapi()
app.connect("127.0.0.1", 7497, 123)
app.nextorderId = None
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

This is the first common beginner failure point: wrong port, client ID conflict, or API disabled can look like “nothing happens.” By waiting for nextValidId, we confirm the socket session is truly usable before sending risk.

Poll until we receive nextValidId, which tells us IB has accepted the connection and issued an order ID we can safely use.

In [ ]:
while True:
    if isinstance(app.nextorderId, int):
        print("connected")
        break
    print("waiting for connection")
    time.sleep(1)

The “order ID gate” is a simple guardrail that prevents silent no-op submissions and makes debugging straightforward. In real systems we’d replace prints with logs, but the principle is the same: don’t trade until you can observe state.

## Place and cancel a test order

Build a basic limit order and submit it with the current nextorderId against an AAPL SMART-routed stock contract.

In [ ]:
order = Order()
order.action = "BUY"
order.totalQuantity = 10
order.orderType = "LMT"
order.lmtPrice = "130.00"

app.placeOrder(app.nextorderId, stock_contract("AAPL"), order)

time.sleep(3)

print("cancelling order")
app.cancelOrder(app.nextorderId, "")

time.sleep(3)
app.disconnect()

Using a limit order (not a market order) is a safer default while you’re validating plumbing, because it constrains the price you’ll pay if something goes wrong. The quick cancel-and-disconnect sequence is also a practical habit for early paper trading: it exercises the full lifecycle (submit, status callbacks, cancel) so we know we can track and control orders end-to-end.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://www.pyquantnews.com/getting-started-with-python-for-quant-finance/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.